# Setup
First we test to make sure that are in fact in the environment that we think we are in.

In [1]:
# Check that the correct runtime is enabled and active
%cd ~/draping_playground/diffsim/pysim
working_path = "/home/jjdunlop/draping_playground/"

!conda info --env
!python --version
!pwd

/home/jjdunlop/draping_playground/diffsim/pysim
# conda environments:
#
base                     /home/jjdunlop/anaconda3
diffsim               *  /home/jjdunlop/anaconda3/envs/diffsim

Python 3.6.13 :: Anaconda, Inc.
/home/jjdunlop/draping_playground/diffsim/pysim


In [2]:
# We can use the JSON library in Python to create a dictionary and then save it to a .json file.

import json

# Define the paths as variables
cloth_mesh_path = "/home/jjdunlop/draping_playground/3D_Models/2d_shape_flatmesh_99pc.obj"
obstacle_mesh_path = "/home/jjdunlop/draping_playground/3D_Models/3d_shape.obj"

# Define the configuration dictionary
config = {
    "frame_time": 0.125,
    "frame_steps": 1,
    "end_time": 10,
    "cloths": [
        {
            "mesh": cloth_mesh_path,
            "transform": {"scale": 0.01, "translate": [-0.1, -0.3, 0.3]},
            "materials": [{"data": "materials/gray-interlock.json",
                           "thicken": 2}],
            "remeshing": {
                "refine_angle": 0.3,
                "refine_compression": 0.005,
                "refine_velocity": 0.5,
                "size": [10e-3, 100e-3],
                "aspect_min": 0.2
            }
        }
    ],
    "obstacles": [
        {
            "mesh": obstacle_mesh_path,
            "transform": {"scale": 0.01, "translate": [-0.1, -0.3, -0.5]},
            "velocity": [0, 0, 0, 0, 0, 0],
            "movable": 0
        }
    ],
    "disable": ["remeshing", "proximity"],
    "handles": [{"nodes": [302, 198, 2, 23]}],
    "gravity": [0, 0, -2],
    "magic": {"repulsion_thickness": 5e-3, "collision_stiffness": 1e6}
}

# Now we write the config to a JSON file
config_file = '/home/jjdunlop/draping_playground/diffsim/pysim/conf/curved_draping.json' # We are writing to the writable directory
with open(config_file, 'w') as f:
    json.dump(config, f, indent=4)

print(config_file)


/home/jjdunlop/draping_playground/diffsim/pysim/conf/curved_draping.json


In [3]:
import trimesh
import trimesh.transformations as tf
import numpy as np

curved_mesh = trimesh.load(obstacle_mesh_path,process=False)
flat_mesh = trimesh.load(cloth_mesh_path,process=False)
scene = trimesh.Scene(base_frame='world')
curved_mesh.apply_scale(0.01)
flat_mesh.apply_scale(0.01)

transform = tf.translation_matrix([-0.1,-0.3,0.3])
flat_mesh.apply_transform(transform)
f_id = scene.add_geometry(flat_mesh)

transform = tf.translation_matrix([-0.1,-0.3,-0.5])
curved_mesh.apply_transform(transform)
c_id = scene.add_geometry(curved_mesh)

handles = [302, 198, 2, 23]
steps = 30

trajectories = []

for handle in handles:
    ray_origin = flat_mesh.vertices[handle]
    
    ray_direction = curved_mesh.vertices[handle] - flat_mesh.vertices[handle]
    ray_direction = ray_direction

    locations, index_ray, index_tri = curved_mesh.ray.intersects_location(
        ray_origins=np.array([ray_origin]),
        ray_directions=np.array([ray_direction]))

    location = locations[0]
    path = np.dstack([np.linspace(ray_origin[0],location[0],steps),np.linspace(ray_origin[1],location[1],steps),np.linspace(ray_origin[2],location[2],steps)])

    path = path.squeeze()
    trajectories.append(path)

trajectories = np.array(trajectories) #this is positions, not vs
trajectories = np.diff(trajectories,axis=1) #vs
trajectories = trajectories.transpose(1,0,2)
trajectories = np.concatenate([trajectories,np.expand_dims(np.zeros_like(trajectories[0]),0)])

This code does the following:

- Sets up an output directory for the simulation results and logs.
- Loads and saves a JSON configuration for the simulation.
- Initializes the simulation environment with specific parameters for the cloth and obstacles.
- Defines functions to reset the simulation at the start of each epoch, run the simulation with the current parameters, and calculate loss based on the cloth's position relative to predefined boundary vertices.
- Uses gradient descent (via the Adam optimizer) to adjust the parameters (param_v) in order to minimize the loss function, thereby optimizing the simulation towards a desired state.
- Logs the progress and results of each epoch to a file, including the loss and optimization parameters.

In [ ]:
!rm -rf diffsim/pysim/curved_draping_default_out
!pwd

import torch
import arcsim
import gc
import time
import json
import gc
import os

from datetime import datetime
now = datetime.now()
timestamp = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')

out_path = '/home/jjdunlop/draping_playground/curved_draping_default_out'

if not os.path.exists(out_path):
  os.mkdir(out_path)

with open(config_file,'r') as f:
  config = json.load(f)

def save_config(config, file):
  with open(file,'w') as f:
    json.dump(config, f)

save_config(config, out_path+'/conf.json')

torch.set_num_threads(24)
spf = config['frame_steps']
frame_time = config['frame_time']
scalev=1

pre_steps = 10
#steps=30
epochs=10

seed_point_index = 292
#handles = [302, 198, 2, 23]
boundary = [  2,   1,  68,  74,  77,  79,  81,  83,  85,  87,  89, 198,  65, 
         63,  61,  59,  57,  55,  53,  51,  49,  47,  45,  43,  41,  39,
         37,  35,  33,  31,  29,  27,  25,  24,  23,  21,  19,  17,  15,
         13,  11,   9,   7,   4,   3, 302,  92,  91,  95,  98,  97, 101,
        103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127,
        129, 131, 133, 139, 141,   0]

def reset_sim(sim, epoch):
  if epoch < epochs:
    arcsim.init_physics(out_path+'/conf.json', out_path+'/out%d'%epoch,False)
  else:
    arcsim.init_physics(out_path+'/conf.json',out_path+'/out',False)

def run_sim(steps,sim,param_v):

  loss = 0.
  
  print("=======================================")
  print("Next Simulation Starting")
  for step in range(pre_steps + steps):
    print(f"step {step + 1}/{pre_steps + steps}", end='\r', flush=True)

    if step > pre_steps:
      for i in range(len(handles)):
        sim.cloths[0].mesh.nodes[handles[i]].v += param_v[step-pre_steps,i] * spf
        #print(sim.cloths[0].mesh.nodes[handles[i]].x)

    # from IPython.core.debugger import set_trace
    # set_trace()

    #print(sim.obstacles[0].curr_state_mesh.nodes[seed_point_index].x - sim.cloths[0].mesh.nodes[seed_point_index].x)
    
    # loss_idxs = [*boundary]

    # if step > pre_steps:
    #   for i in loss_idxs:
    #     loss += torch.norm(sim.obstacles[0].curr_state_mesh.nodes[i].x - sim.cloths[0].mesh.nodes[i].x, p=1) #* (1 + ((step-pre_steps)/steps))

    arcsim.sim_step()

  loss_idxs = [*boundary]
  for i in loss_idxs:
    loss += torch.norm(sim.obstacles[0].curr_state_mesh.nodes[i].x - sim.cloths[0].mesh.nodes[i].x, p=1)

  v_limit = 0.5 #was 0.25
  stiffness = 2
  weight = 2/30 * steps * 1/7 * len(loss_idxs)
  v_constraint = torch.sum(weight / (1.0+torch.exp(stiffness*(1.0 - (torch.abs(param_v)/v_limit)))))

  avg = torch.mean(param_v,dim=1,keepdim=True)
  reg = (torch.norm(param_v-avg, dim=2)**2).mean()

  total_loss = loss #+ reg #+ v_constraint

  return total_loss

def do_train(cur_step,optimizer,scheduler,sim,param_v):
  epoch = 0
  while True:
    reset_sim(sim, epoch)
    st = time.time()
    loss = run_sim(steps, sim, param_v)
    en0 = time.time()
    optimizer.zero_grad()

    loss.backward(retain_graph=True)

    en1 = time.time()
    #print(param_v.data)
    #print(param_v.grad.data)
    f.write('epoch {}:  loss={} \n'.format(epoch,  loss.data))
    print('epoch {}:  loss={} \n'.format(epoch, loss.data))

    print('forward time={}'.format(en0-st))
    print('backward time={}'.format(en1-en0))

    #param_v.grad.data.clamp_(-25,25)

    optimizer.step()

    arcsim.delete_mesh(sim.cloths[0].mesh)

    scheduler.step(loss.item())

    if epoch>=epochs:
      break
    epoch = epoch + 1

with open(out_path+('/log%s.txt'%timestamp),'w',buffering=1) as f:
  tot_step = 1
  sim=arcsim.get_sim()
  # reset_sim(sim)

  #param_v = torch.zeros([steps, 4,3],dtype=torch.float64, requires_grad=True)
  param_v = torch.tensor(trajectories/frame_time,dtype=torch.float64, requires_grad=True)

  optimizer = torch.optim.Adam([param_v],lr=0.01)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=0, verbose=True)

  for cur_step in range(tot_step):
    do_train(cur_step,optimizer,scheduler,sim,param_v)

print("done")

/home/jjdunlop/draping_playground/diffsim/pysim
Next Simulation Starting
epoch 0:  loss=13.348705828652356 

forward time=38.415716886520386
backward time=38.864098072052
Next Simulation Starting
epoch 1:  loss=11.184798473880434 

forward time=46.63276791572571
backward time=41.33958864212036
Next Simulation Starting
epoch 2:  loss=10.424213846119091 

forward time=47.94053840637207
backward time=42.01633048057556
Next Simulation Starting


In [8]:
%%writefile visualize.py
import torch
import arcsim

with torch.autograd.profiler.profile() as prof:
	  arcsim.msim(4,['arcsim','replay','/home/jjdunlop/draping_playground/curved_draping_default_out/out', '0.0'])
print(prof)

Writing visualize.py
